In [1]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np
import mlflow

In [2]:
train = pd.read_csv('../data/train.csv')

In [3]:
X = train.drop(columns=[
    'ID', 'target1', 'target2', 'target3', 'target4', 'target5',
    'target6', 'target7', 'target8', 'target9', 'target10', 'target11'
])

y = train.drop(columns=[
    'ID', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
    'feature7', 'feature8', 'feature9', 'feature10', 'feature11',
    'feature12', 'feature13', 'feature14', 'feature15'
])


In [4]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

Hyper-param optmization

In [5]:
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, KFold
import mlflow
import numpy as np
import warnings
from sklearn.linear_model import Ridge




warnings.filterwarnings('ignore')

def objective(trial):
    with mlflow.start_run(run_name=f'Trial - {trial.number}', nested=True):
        # único "hiperparâmetro" do LinearRegression
                # dentro do objective:
        params = {
            'alpha': trial.suggest_float('alpha', 1e-6, 10.0, log=True),
            'fit_intercept': True,
            'random_state': 42
        }
        mlflow.log_params(params)
        model = Ridge(**params)
        # e o restante é igual

        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        # scikit-learn >=1.0 tem scoring 'neg_root_mean_squared_error'
        scores = cross_val_score(
            model, X, y,
            scoring='neg_root_mean_squared_error',
            cv=kf
        )
        rmse = -scores.mean()
        mlflow.log_metric("rmse", rmse)

        return rmse

# Exemplo de uso:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=10)


c:\Users\Guilherme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(seed=42),
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=20,
        n_warmup_steps=10,
        interval_steps=5
    )
)

mlflow.set_tracking_uri('http://127.0.0.1:5000')
mlflow.set_experiment('DSWA - Competicao3')
with mlflow.start_run(run_name='Linear Regression optimization'):
    study.optimize(objective, n_trials=50)
    
    print("Melhores hiperparâmetros:")
    print(study.best_params)
    print(f"Melhor RMSE: {study.best_value:.4f}")

    # Loga os melhores parâmetros e métrica no MLflow
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_rmse", study.best_value)


[I 2025-07-27 17:21:42,777] A new study created in memory with name: no-name-3125e739-eeeb-4740-bb13-e330b9fdd494
[I 2025-07-27 17:21:43,163] Trial 0 finished with value: 0.5916354365724268 and parameters: {'alpha': 0.0004185822729546974}. Best is trial 0 with value: 0.5916354365724268.
[I 2025-07-27 17:21:43,317] Trial 1 finished with value: 0.5943263595950357 and parameters: {'alpha': 4.518560951024111}. Best is trial 0 with value: 0.5916354365724268.


🏃 View run Trial - 0 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/ef6696ba9d43456f8c89ae48fb319642
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
🏃 View run Trial - 1 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/e8247708fed544e6b63ec403fec35d8b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:21:43,472] Trial 2 finished with value: 0.5952192107716853 and parameters: {'alpha': 0.1330324510152292}. Best is trial 0 with value: 0.5916354365724268.
[I 2025-07-27 17:21:43,627] Trial 3 finished with value: 0.593588301703565 and parameters: {'alpha': 0.015509913987594319}. Best is trial 0 with value: 0.5916354365724268.


🏃 View run Trial - 2 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/a136605fcb8d4394b30a7dc9206f4203
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
🏃 View run Trial - 3 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/474211bf44874d909be852710e4bb07b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:21:43,782] Trial 4 finished with value: 0.5922842251368544 and parameters: {'alpha': 1.2363188277052228e-05}. Best is trial 0 with value: 0.5916354365724268.
[I 2025-07-27 17:21:43,935] Trial 5 finished with value: 0.5922842355704422 and parameters: {'alpha': 1.2358382772306929e-05}. Best is trial 0 with value: 0.5916354365724268.


...

🏃 View run Trial - 45 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/d9409fbf60f24e5b9dc23630388af105
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
🏃 View run Trial - 46 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/884f7f49cf19461a858833b5ef23481a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:21:50,690] Trial 47 finished with value: 0.591695966375936 and parameters: {'alpha': 0.004260543887658976}. Best is trial 32 with value: 0.591130458981255.
[I 2025-07-27 17:21:50,856] Trial 48 finished with value: 0.594859788524526 and parameters: {'alpha': 0.05568690718275339}. Best is trial 32 with value: 0.591130458981255.


🏃 View run Trial - 47 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/11d88a703209400989b56327cbd99d89
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
🏃 View run Trial - 48 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/df640213ec56404a92e1b70288ba793c
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732


[I 2025-07-27 17:21:51,035] Trial 49 finished with value: 0.5928852174668885 and parameters: {'alpha': 0.009690722746900563}. Best is trial 32 with value: 0.591130458981255.


🏃 View run Trial - 49 at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/f5b1ca4b858c45b6afeabd5c3dbb401b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
Melhores hiperparâmetros:
{'alpha': 0.001632253674509904}
Melhor RMSE: 0.5911
🏃 View run Linear Regression optimization at: http://127.0.0.1:5000/#/experiments/962208812792484732/runs/66615588c03b473483a81964b6ba6d48
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/962208812792484732
